In [1]:
import pandas as pd
import numpy as np
from nltk import RegexpTokenizer
import multiprocessing
import time
import gc
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
gc.collect()

0

In [4]:
def getIdsRefCves(file):
#     df = pd.read_csv(file, names=['IdPost', 'Content', 'Heading'])
    df = pd.read_csv(file, header=0, usecols=['IdPost', 'Content', 'Heading'])
#     df.IdPost = df.IdPost.astype('int32')
    df['Content'] = df['Content'].astype(str).str.lower()
    df['Heading'] = df['Heading'].astype(str).str.lower()
    df['nCvesContent'] = df['Content'].map(lambda t: len(cves_tkr.tokenize(t)))
    df['nCvesHeading'] = df['Heading'].map(lambda t: len(cves_tkr.tokenize(t)))

    pt_ids = np.array(df[(df['nCvesContent'] > 0) | (df['nCvesHeading'] > 0)]['IdPost'])
    p_ids = np.array(df[df['nCvesContent'] > 0]['IdPost'])
    t_ids = np.array(df[df['nCvesHeading'] > 0]['IdPost'])
    del df
    return pt_ids, p_ids, t_ids

In [5]:
files = ['../Antichat/antichat.csv']
cves_tkr = RegexpTokenizer(r"cve-\d+-\d+|cve-\d+")

pt_ids = np.array([])
p_ids = np.array([])
t_ids = np.array([])
i = 1
for file in files:    
    gc.collect()
    start = time.time()
    pt_ids_, p_ids_, t_ids_ = getIdsRefCves(file)
#     pool = multiprocessing.Pool(1)
#     res = pool.apply(getIdsRefCves, [file])
    print("File: {} demorou {} segundos".format(i,time.time()-start))
    pt_ids = np.concatenate((pt_ids, pt_ids_))
    p_ids = np.concatenate((p_ids_, p_ids))
    t_ids = np.concatenate((t_ids_, t_ids))
    i+=1    

File: 1 demorou 29.5484721660614 segundos


In [6]:
print("#Posts: ", len(p_ids))
print("#Threads: ", len(t_ids))
print("#Posts and #Threads: ", len(pt_ids))

#Posts:  310
#Threads:  132
#Posts and #Threads:  419


In [7]:
len(set(p_ids))

310

In [ ]:
posts = set(p_ids)
threads = set(t_ids)

venn2([posts, threads], set_labels = ('Posts citing CVEs', "Posts not citing CVEs\nbut with its Thread's heading doing it"))
# plt.savefig("venn_posts_headings")

In [8]:
def floatListToStr(l):
    return ", ".join([str(int(n)) for n in l])

In [9]:
f = open("all_posts_with_cves.txt", "w")
f.write(floatListToStr(pt_ids))
# f.write(intListToStr(threads.intersection(posts)))
f.close()

In [9]:
# get what really matters: threads ids
# with open('../Antichat/threads.txt', 'w') as f:
#     for i in df['IdThread'].unique():
#         f.write(str(i) + ", ")